In [1]:
def parseLine(line):
    words = line.split()
    tokens,tags = [],[]
    for word in words:
        try:
            params = word.split("_")
            tokens.append(params[0])
            tags.append(params[1])
        except IndexError as e:
            e
            #dirtydata
            #print("index error, skip data")
    return (tokens,tags)

In [2]:
line = "victorian_BRAND order_BRAND of_BRAND nurses_BRAND lemarchant_STREET rd_STREET"

In [3]:
(tokens,tags) = parseLine(line)

In [4]:
tokens

['victorian', 'order', 'of', 'nurses', 'lemarchant', 'rd']

In [5]:
tags

['BRAND', 'BRAND', 'BRAND', 'BRAND', 'STREET', 'STREET']

In [6]:
import numpy as np

In [7]:
sequences,sequences_tags = [],[]
fh = open('can_test.txt')
for line in fh:
    tokens,tags = parseLine(line)
    if(len(tokens) != 0):
        sequences.append(np.array(tokens))
        sequences_tags.append(np.array(tags))
fh.close()



In [8]:
print(sequences[5])
print(sequences_tags[5])

['nancy' 'ave']
['STREET' 'STREET']


In [9]:
from sklearn.model_selection import train_test_split

In [10]:
train_test_split?

In [11]:
(train_sequences, 
 test_sequences, 
 train_tags, 
 test_tags) = train_test_split(sequences, sequences_tags, train_size=0.8,test_size=0.2,random_state=42,shuffle=True)

In [12]:
#from sklearn.model_selection import train_test_split
 
# 
#(train_sentences, 
# test_sentences, 
## test_tags) = train_test_split(sequences, sequences_tags, test_size=0.2)
#train_test_spli?

 

In [13]:
words, tags = set([]), set([])
 
for s in train_sequences:
    for w in s:
        words.add(w.lower())

for ts in train_tags:
    for t in ts:
        tags.add(t)
        
word2index = {w: i + 2 for i, w in enumerate(list(words))}
word2index['-PAD-'] = 0  # The special value used for padding
word2index['-OOV-'] = 1  # The special value used for OOVs
 
tag2index = {t: i + 1 for i, t in enumerate(list(tags))}
tag2index['-PAD-'] = 0  # The special value used to padding

In [14]:
train_sequences_X, test_sequences_X, train_tags_y, test_tags_y = [], [], [], []
 
for s in train_sequences:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    train_sequences_X.append(s_int)

for s in test_sequences:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    test_sequences_X.append(s_int)

for s in train_tags:
        try:
            train_tags_y.append([tag2index[t] for t in s])
        except:
            train_tags_y.append([0])

for s in test_tags:
        try:
            test_tags_y.append([tag2index[t] for t in s])
        except:
            test_tags_y.append([0])
        
print(train_sequences_X[0])
print(test_sequences_X[0])
print(train_tags_y[0])
print(test_tags_y[0])

[7, 6, 2]
[6, 2, 7]
[2, 1, 1]
[1, 1, 2]


In [15]:
print(len(train_sequences_X))
print(len(test_sequences_X))

print(len(train_tags_y))
print(len(test_tags_y))

10
3
10
3


In [16]:
MAX_LENGTH = len(max(train_sequences_X, key=len))
print(MAX_LENGTH)  # 271

3


In [17]:
from keras.preprocessing.sequence import pad_sequences
 
train_sequences_X = pad_sequences(train_sequences_X, maxlen=MAX_LENGTH, padding='post')
test_sequences_X = pad_sequences(test_sequences_X, maxlen=MAX_LENGTH, padding='post')
train_tags_y = pad_sequences(train_tags_y, maxlen=MAX_LENGTH, padding='post')
test_tags_y = pad_sequences(test_tags_y, maxlen=MAX_LENGTH, padding='post')
 
print(train_sequences_X[0])
print(test_sequences_X[0])
print(train_tags_y[0])
print(test_tags_y[0])

Using TensorFlow backend.


[7 6 2]
[6 2 7]
[2 1 1]
[1 1 2]


In [18]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation
from keras.optimizers import Adam
 
 
model = Sequential()
model.add(InputLayer(input_shape=(MAX_LENGTH, )))
model.add(Embedding(len(word2index), 128))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(TimeDistributed(Dense(len(tag2index))))
model.add(Activation('softmax'))
 
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy'])
 
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 3, 128)            1408      
_________________________________________________________________
bidirectional_1 (Bidirection (None, 3, 512)            788480    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 3, 3)              1539      
_________________________________________________________________
activation_1 (Activation)    (None, 3, 3)              0         
Total params: 791,427
Trainable params: 791,427
Non-trainable params: 0
_________________________________________________________________


In [19]:
def to_categorical(sequences, categories):
    cat_sequences = []
    for s in sequences:
        cats = []
        for item in s:
            cats.append(np.zeros(categories))
            cats[-1][item] = 1.0
        cat_sequences.append(cats)
    return np.array(cat_sequences)

In [20]:
cat_train_tags_y = to_categorical(train_tags_y, len(tag2index))
print(cat_train_tags_y[0])

[[0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [21]:
model.fit(train_sequences_X, to_categorical(train_tags_y, len(tag2index)), batch_size=1024, epochs=3, validation_split=0.2)
model.save(filepath="rnn.model")

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 8 samples, validate on 2 samples
Epoch 1/3
8/8 [==============================] - 2s 195ms/step - loss: 1.1007 - acc: 0.1250 - val_loss: 1.0846 - val_acc: 0.6667
Epoch 2/3
8/8 [==============================] - 0s 3ms/step - loss: 1.0857 - acc: 0.6667 - val_loss: 1.0712 - val_acc: 0.6667
Epoch 3/3
8/8 [==============================] - 0s 3ms/step - loss: 1.0706 - acc: 0.6667 - val_loss: 1.0572 - val_acc: 0.6667


In [22]:
scores = model.evaluate(test_sequences_X, to_categorical(test_tags_y, len(tag2index)))
print(f"{model.metrics_names[1]}: {scores[1] * 100}")   # acc: 99.09751977804825

3/3 [==============================] - 0s 654us/step
acc: 66.66666865348816


In [24]:
test_tags_y = to_categorical(test_tags_y, len(tag2index))

In [29]:
type(test_tags_y)

numpy.ndarray

In [30]:
type(test_sequences_X)

numpy.ndarray

In [109]:
test_samples = [
    "king street".split(),
    "wayfare restauran main st 311".split(),
    "falls cadillac pres de la thorold stone rd".split()
]
print(test_samples)

[['king', 'street'], ['wayfare', 'restauran', 'main', 'st', '311'], ['falls', 'cadillac', 'pres', 'de', 'la', 'thorold', 'stone', 'rd']]


In [110]:
test_samples_X = []
for s in test_samples:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
    test_samples_X.append(s_int)
 
test_samples_X = pad_sequences(test_samples_X, maxlen=MAX_LENGTH, padding='post')
print(test_samples_X)

[[ 52117  64495      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0]
 [     1      1  23203   3147  92094      0      0      0      0      0
       0      0      0      0      0      0      0      0]
 [100704  62859  99304  60819  39226 105553  71431  98718      0      0
       0      0      0      0      0      0      0      0]]


In [111]:
predictions = model.predict(test_samples_X)
print(predictions, predictions.shape)

[[[2.44228033e-07 1.85927883e-01 1.87560431e-06 6.38416503e-04
   8.13339353e-01 9.22444888e-05]
  [1.28117623e-04 4.80535746e-01 2.49975960e-06 7.93299172e-04
   5.18474996e-01 6.53061506e-05]
  [9.98677790e-01 1.28250110e-06 8.40640908e-08 1.13478210e-03
   1.85615994e-04 3.11253928e-07]
  [9.99999642e-01 1.70752071e-13 2.87434833e-11 2.49423550e-07
   1.22943689e-07 9.34411437e-11]
  [1.00000000e+00 5.36143963e-17 5.15426731e-13 8.86929641e-09
   1.81319426e-09 1.59441752e-12]
  [1.00000000e+00 4.20488860e-17 9.67074185e-13 1.14477476e-08
   3.41589157e-09 2.77487235e-12]
  [1.00000000e+00 8.99147179e-17 3.41162818e-12 2.65833755e-08
   1.16623173e-08 7.38388257e-12]
  [1.00000000e+00 1.12615823e-16 6.96862056e-12 4.48944775e-08
   2.28640857e-08 9.85467871e-12]
  [1.00000000e+00 7.99053295e-17 7.98133781e-12 4.86602332e-08
   2.51160746e-08 6.94465752e-12]
  [1.00000000e+00 4.07320178e-17 6.28681655e-12 3.52419782e-08
   1.85692333e-08 3.48387009e-12]
  [1.00000000e+00 1.92867384e-

In [112]:
def logits_to_tokens(sequences, index):
    token_sequences = []
    for categorical_sequence in sequences:
        token_sequence = []
        for categorical in categorical_sequence:
            token_sequence.append(index[np.argmax(categorical)])
 
        token_sequences.append(token_sequence)
 
    return token_sequences

In [66]:
print(logits_to_tokens(predictions, {i: t for t, i in tag2index.items()}))

[['STREET', 'STREET', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-'], ['BRAND', 'BRAND', 'STREET', 'STREET', 'DOOR', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-'], ['BRAND', 'BRAND', 'CONJ', 'CONJ', 'CONJ', 'STREET', 'STREET', 'STREET', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-']]


In [61]:
model.save("onebox_tagging.rnn")

In [107]:
def tagInput(input):
    tokens = input.split()
    test_samples_Xx = []
    s_intx = []
  
    for w in tokens:
        try:
            s_intx.append(word2index[w.lower()])
        except KeyError:
            s_intx.append(word2index['-OOV-'])
        test_samples_Xx.append(s_intx)
    print("before test samples "+str(test_samples_Xx))
    test_samples_Xx = pad_sequences(test_samples_Xx, maxlen=MAX_LENGTH, padding='post')
    print("test samples "+str(test_samples_Xx))
    predictionsx = model.predict(test_samples_Xx)
    tags = logits_to_tokens(predictionsx, {i: t for t, i in tag2index.items()})
   
    return tags
    

In [108]:
tagInput(input = "falls cadillac pres de la thorold stone rd")

before test samples [[100704, 62859, 99304, 60819, 39226, 105553, 71431, 98718], [100704, 62859, 99304, 60819, 39226, 105553, 71431, 98718], [100704, 62859, 99304, 60819, 39226, 105553, 71431, 98718], [100704, 62859, 99304, 60819, 39226, 105553, 71431, 98718], [100704, 62859, 99304, 60819, 39226, 105553, 71431, 98718], [100704, 62859, 99304, 60819, 39226, 105553, 71431, 98718], [100704, 62859, 99304, 60819, 39226, 105553, 71431, 98718], [100704, 62859, 99304, 60819, 39226, 105553, 71431, 98718]]
test samples [[100704  62859  99304  60819  39226 105553  71431  98718      0      0
       0      0      0      0      0      0      0      0]
 [100704  62859  99304  60819  39226 105553  71431  98718      0      0
       0      0      0      0      0      0      0      0]
 [100704  62859  99304  60819  39226 105553  71431  98718      0      0
       0      0      0      0      0      0      0      0]
 [100704  62859  99304  60819  39226 105553  71431  98718      0      0
       0      0      

[['BRAND',
  'BRAND',
  'CONJ',
  'CONJ',
  'CONJ',
  'STREET',
  'STREET',
  'STREET',
  '-PAD-',
  '-PAD-',
  '-PAD-',
  '-PAD-',
  '-PAD-',
  '-PAD-',
  '-PAD-',
  '-PAD-',
  '-PAD-',
  '-PAD-'],
 ['BRAND',
  'BRAND',
  'CONJ',
  'CONJ',
  'CONJ',
  'STREET',
  'STREET',
  'STREET',
  '-PAD-',
  '-PAD-',
  '-PAD-',
  '-PAD-',
  '-PAD-',
  '-PAD-',
  '-PAD-',
  '-PAD-',
  '-PAD-',
  '-PAD-'],
 ['BRAND',
  'BRAND',
  'CONJ',
  'CONJ',
  'CONJ',
  'STREET',
  'STREET',
  'STREET',
  '-PAD-',
  '-PAD-',
  '-PAD-',
  '-PAD-',
  '-PAD-',
  '-PAD-',
  '-PAD-',
  '-PAD-',
  '-PAD-',
  '-PAD-'],
 ['BRAND',
  'BRAND',
  'CONJ',
  'CONJ',
  'CONJ',
  'STREET',
  'STREET',
  'STREET',
  '-PAD-',
  '-PAD-',
  '-PAD-',
  '-PAD-',
  '-PAD-',
  '-PAD-',
  '-PAD-',
  '-PAD-',
  '-PAD-',
  '-PAD-'],
 ['BRAND',
  'BRAND',
  'CONJ',
  'CONJ',
  'CONJ',
  'STREET',
  'STREET',
  'STREET',
  '-PAD-',
  '-PAD-',
  '-PAD-',
  '-PAD-',
  '-PAD-',
  '-PAD-',
  '-PAD-',
  '-PAD-',
  '-PAD-',
  '-PAD-'],
 ['BR